In [26]:
%reload_ext autoreload
%autoreload 2

import os, sys

import numpy as np
from scipy import interpolate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc_file('../matplotlibrc')

In [10]:
if os.environ['USER'] == 'yitians' and 'submit' in os.uname().nodename:
    os.environ['DH_DIR'] = '/work/submit/yitians/darkhistory/DarkHistory'
    os.environ['DM21CM_DATA_DIR'] = '/data/submit/yitians/DM21cm'
sys.path.append(os.environ['DH_DIR'])
sys.path.append('..')

In [18]:
import main

import dm21cm.physics as phys

## DM turns on

In [39]:
run_name = 'rundh_turnon_z40'
# DM turns on
injection_modifier_func = lambda rs: float(rs < 41)
# custom xe levels
base_records = np.load(f'../data/run_info/base_records.npy', allow_pickle=True).item()
def xe_reion_func(rs):
    return (1 + phys.chi) * interpolate.interp1d(1 + base_records['z'], base_records['x_e'])(rs)

reion_rs = 41
end_rs = base_records['z'][-1] + 1

In [41]:
xe_reion_func(8)

0.8089399659344682

In [38]:
soln = main.evolve(
    DM_process='swave', mDM=1e10, sigmav=1e-23, primary='phot_delta',
    start_rs=3000, end_rs=end_rs,
    coarsen_factor=12, backreaction=True, reion_switch=True,
    struct_boost=phys.struct_boost_func('erfc 1e-3'),
    verbose=1, injection_modifier_func=injection_modifier_func,
    xe_reion_func=xe_reion_func, reion_rs=reion_rs
)

Loading time: 0.003 s





  0%|          | 0/506 [00:00<?, ?it/s]

Initialization time: 0.016 s





  0%|          | 2/506 [00:01<04:47,  1.75it/s]


  1%|          | 3/506 [00:02<06:51,  1.22it/s]


  1%|          | 4/506 [00:03<07:40,  1.09it/s]


  1%|          | 5/506 [00:04<08:38,  1.03s/it]


  1%|          | 6/506 [00:05<08:55,  1.07s/it]


  1%|▏         | 7/506 [00:07<09:29,  1.14s/it]


  2%|▏         | 8/506 [00:08<09:52,  1.19s/it]


  2%|▏         | 9/506 [00:09<10:05,  1.22s/it]


  2%|▏         | 10/506 [00:10<10:00,  1.21s/it]


  2%|▏         | 11/506 [00:12<10:32,  1.28s/it]


  2%|▏         | 12/506 [00:13<10:10,  1.23s/it]


  3%|▎         | 13/506 [00:14<09:53,  1.20s/it]


  3%|▎         | 14/506 [00:15<09:42,  1.18s/it]


  3%|▎         | 15/506 [00:16<09:34,  1.17s/it]


  3%|▎         | 16/506 [00:17<08:58,  1.10s/it]


  3%|▎         | 17/506 [00:19<09:31,  1.17s/it]


  4%|▎         | 18/506 [00:20<09:11,  1.13s/it]


  4%|▍         | 19/506 [00:21<09:04,  1.12s/it]


  4%|▍         | 20/506 [00:22<08:29,  1.05s/it]


  4%|▍         | 21/506 [00:23<09:20

ValueError: zero-size array to reduction operation minimum which has no identity

In [15]:
pickle.dump(soln, open(f'../data/{run_name}_soln.p', 'wb'))

In [16]:
arr_records = {
    'z' : soln['rs'] - 1,
    'x_e' : soln['x'][:,0],
    'T_k' : soln['Tm']/phys.kB,
    'f_heat' : soln['f']['low']['heat'] + soln['f']['high']['heat'],
    'f_ion' : soln['f']['low']['H ion']  + soln['f']['high']['H ion'] + \
              soln['f']['low']['He ion'] + soln['f']['high']['He ion'],
    'f_exc' : soln['f']['low']['exc'] + soln['f']['high']['exc'],
}

z_low, z_high = 6, 44
z_arr = arr_records['z'][::-1]
i_low = np.searchsorted(z_arr, z_low)
i_high = np.searchsorted(z_arr, z_high)

for k in arr_records.keys():
    arr_records[k] = arr_records[k][::-1][i_low:i_high]

In [17]:
np.save(f'../data/run_info/{run_name}_records', arr_records)